# 安徽安庆市项目`WRF-CMAQ`模拟分析
## 观测结果预处理

---
*@author: Evan*\
*@date: 2023-03-28*

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from matplotlib import rcParams
config = {
    "font.family":'Times New Roman',
    "mathtext.fontset":'stix',
    "font.serif": ['SimSun'],
}
rcParams.update(config)

import cartopy.crs as ccrs
import cartopy.feature as cfeat
from cartopy.io.shapereader import Reader

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

In [28]:
# 读取原始数据
xls=pd.read_excel('F:/Data/case_anqing/April/obsdata/originaldata.xlsx',header=0,na_values=['—',-99])

# 将日期与时间列合并
xls['datetime'] = pd.to_datetime(xls['日期'].astype(str) + ' ' + xls['时间'].astype(str))

# 删除“日期”列和“时间”列
xls.drop(['日期', '时间'], axis=1, inplace=True)

# 设置为索引
xls.set_index('datetime',inplace=True)
xls

,城市,市县,站点,SO2,NO2,NOx,NO,CO,O3,PM10,PM2.5,风速(m/s),风向(°),气压(hPa),气温(℃),湿度(%),降水量(mm)
datetime,,,,,,,,,,,,,,,,,
2023-04-01 00:00:00,安庆市,宜秀区,安庆大学,9,52,56,3,0.833,28,44,27,0.6,316.0,1010.5,14.4,80.0,0.0
2023-04-01 01:00:00,安庆市,宜秀区,安庆大学,8,35,36,1,0.816,40,45,31,0.4,350.9,1010.3,13.8,84.0,0.0
2023-04-01 02:00:00,安庆市,宜秀区,安庆大学,8,36,37,1,0.819,40,51,35,0.7,25.8,1010.1,13.5,86.0,0.0
2023-04-01 03:00:00,安庆市,宜秀区,安庆大学,9,30,32,1,0.812,35,61,38,0.4,14.5,1009.5,13.0,88.0,0.0
2023-04-01 04:00:00,安庆市,宜秀区,安庆大学,8,35,36,1,0.824,27,56,39,0.4,28.9,1009.1,12.9,89.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-30 19:00:00,安庆市,宜秀区,市人大,12,13,15,1,0.412,121,121,35,1.1,206.7,1008.0,22.5,38.0,NaN
2023-04-30 20:00:00,安庆市,宜秀区,市人大,13,14,15,1,0.486,115,98,35,0.8,204.5,1008.6,21.4,40.0,NaN
2023-04-30 21:00:00,安庆市,宜秀区,市人大,15,22,24,1,0.585,98,109,42,0.8,182.1,1009.3,20.5,48.0,NaN


In [29]:
# 将所有因包含括号等字符被读取为字符串的数据设置为空值
xls = xls.replace(to_replace=[r'\(.*?\)'], value=np.nan, regex=True)

In [14]:
xls[xls['站点']=='安庆政务中心']['气压(hPa)']

datetime
2023-04-01 00:00:00      NaN
2023-04-01 01:00:00    101.3
2023-04-01 02:00:00    101.3
2023-04-01 03:00:00    101.2
2023-04-01 04:00:00    101.2
                       ...  
2023-04-30 19:00:00    101.2
2023-04-30 20:00:00    101.3
2023-04-30 21:00:00    101.3
2023-04-30 22:00:00    101.4
2023-04-30 23:00:00    101.4
Name: 气压(hPa), Length: 720, dtype: float64

In [30]:
# 发现安庆政务中心站点的气压数据整体小一个量级
data_replace=xls[xls['站点'].isin(['安庆政务中心'])]['气压(hPa)']
xls['气压(hPa)'].replace(data_replace.values,data_replace.values*10,inplace=True)

In [31]:
# 根据站点分组导出
grouped = xls.groupby('站点')
t_index=pd.date_range('2023-04-01T00','2023-04-30T23',freq='h')

for group_name, group_df in grouped:
    # 时间序列如有缺失，则补充空值占位
    group_df.reindex(t_index)
    group_df.to_excel(f'D:/Download/{group_name}.xlsx', index=True)

In [32]:
# 将全部站点取平均并导出
xls.groupby(xls.index).mean().to_excel('D:/Download/allsite.xlsx',index=True)

In [33]:
# 将城区站与通道站分开
urban_path='F:/Data/case_anqing/April/obsdata/urban/'
channel_path='F:/Data/case_anqing/April/obsdata/channel/'
other_path='F:/Data/case_anqing/April/obsdata/other/'
urban_files = [os.path.splitext(filename)[0] for filename in os.listdir(urban_path) if filename.endswith('.xlsx')]
channel_files = [os.path.splitext(filename)[0] for filename in os.listdir(channel_path) if filename.endswith('.xlsx')]
other_files = [os.path.splitext(filename)[0] for filename in os.listdir(other_path) if filename.endswith('.xlsx')]

urban=xls[xls['站点'].isin(urban_files)]
channel=xls[xls['站点'].isin(channel_files)]
other=xls[xls['站点'].isin(other_files)]

# 平均并导出
urban.groupby(urban.index).mean().to_excel('D:/Download/urban.xlsx',index=True)
channel.groupby(channel.index).mean().to_excel('D:/Download/channel.xlsx',index=True)
other.groupby(other.index).mean().to_excel('D:/Download/other.xlsx',index=True)